# Aggregate Data

Check with Kim if this is needed and what exactly makes sense. 


*Kim's list of variables*

*in addition: proportion specificity*


In [25]:
import csv
from collections import defaultdict
import pandas as pd
from collections import Counter
import os

In [ ]:
#RdC_16feb2023_Bo

In [26]:

ref_levels = [
    "individual-TG-levels",
    "middle-TG-levels",
    "high-TG-levels",
    "supra-TG-levels"
]


In [113]:
# data[0]

In [27]:

def get_cat_info(target_category_dicts, target_labels, label_id):
    
    d = target_category_dicts[0]
    
    labels = []
    for l in target_labels:
        val = d[l].split(' ')
        for v in val:
            if v == label_id:
                labels.append(l)
    return ' '.join(labels)




def load_data(test_path):
    
    try:
        with open(test_path) as infile:
            data_original = list(csv.DictReader(infile, delimiter = ','))
    except FileNotFoundError:
        print('file not found')
        data_original = []
    return data_original
    
    

def process_file(test_path,  ref_levels, atricle_id):

    data_out = []
    
    data_original = load_data(test_path)
    
    label_token_dicts = defaultdict(list)
    label_coref_dict = dict()
    label_behav_mapping = defaultdict(set)
        

    for d in data_original:
        if d['TC-reference'] != '':
            for label_id in d['TC-reference'].split(' '):
                label_token_dicts[label_id].append(d)
        if d['Behav-predicate'] != '':
            for label_id in d['Behav-predicate'].split(' '):
                if 'IkD2_fEYyH' == label_id:
                    print('Found problematic label')
                label_token_dicts[label_id].append(d)


        if d['ref-id-relations'] != '':
            coref_ids = d['ref-id-relations'].split(' ')
            for coref_id in coref_ids:
                coref_list = coref_id.split('-_-')
                if len(coref_list) == 2:
                    coref_chain, label_id = coref_list
                    label_coref_dict[label_id] = coref_chain
                else:
                    print('problem')
                    print(coref_id)
                    print(coref_list)
            #     print(coref_chain, label_id)
            # print()

        # get mappings ref behav
        if d['behav-id-mapping'] != '':
            for ref_behav_mapping in d['behav-id-mapping'].split(' '):
                label_id1, label_id2 = ref_behav_mapping.split('-_-')
                label_behav_mapping[label_id1].add(label_id2)
                label_behav_mapping[label_id2].add(label_id1)
        
        
    for label, token_dicts in label_token_dicts.items():
        tokens = ' '.join([d['token'] for d in token_dicts])
        if label in label_coref_dict:
            # print(token_dicts[0].keys())
            

            if label in label_behav_mapping:
                # print('Description found')
                behav_labels = label_behav_mapping[label]
                for behav_label in behav_labels:
                    
                    behav_token_dicts = label_token_dicts[behav_label]
                    if len(behav_token_dicts) == 0:
                        print('Problem')
                        print(behav_label)
                        print(behav_token_dicts)
                    tokens_behav = ' '.join([d['token'] for d in behav_token_dicts])
                    
                    d_out = dict()
                    coref = label_coref_dict[label]
                    # d_out['Reference'] = tokens
                    # d_out['coref'] = coref

                    # d_out['Newspaper_id'] = news_id
                    # d_out['Lstudio_id'] = ls_id
                    d_out['Article_id'] = article_id
                    # d_out['Topic'] = topic

                    d_out['Referent_id'] = coref
                    d_out['Reference_text'] = tokens
                    #get_cat_info(target_category_dicts, target_labels, label_id)
                    d_out['Reference_level'] = get_cat_info(token_dicts, ref_levels, label)
                    # d_out['Reference_valence'] =  get_cat_info(token_dicts, valence_ref, label)
                    # d_out["Target's_role"] = get_cat_info(token_dicts, roles, label )
                    # d_out['Group_membership'] = get_cat_info(token_dicts, group_ref, label)
                    # d_out['Reference_demographics'] = get_cat_info(token_dicts, ref_dem, label)
                    # # print('Description', tokens_behav)
                    # d_out['Description_text'] = tokens_behav
                    # d_out['Description_level'] = get_cat_info(behav_token_dicts, lcm_level, behav_label)
                    # d_out['Description_valence'] = get_cat_info(behav_token_dicts, valence_beh, behav_label)
                    # d_out['Description_group'] = get_cat_info(behav_token_dicts, group_beh, behav_label)
                    # d_out['Description_demographics'] = get_cat_info(behav_token_dicts, des_dem, behav_label)
                    data_out.append(d_out)
            elif label not in label_behav_mapping:
                d_out = dict()
                coref = label_coref_dict[label]
                # d_out['Reference'] = tokens
                # d_out['coref'] = coref

                # d_out['Newspaper_id'] = news_id
                # d_out['Lstudio_id'] = ls_id
                d_out['Article_id'] = article_id
                # d_out['Topic'] = topic

                d_out['Referent_id'] = coref
                d_out['Reference_text'] = tokens
                d_out['Reference_level'] = get_cat_info(token_dicts, ref_levels, label)
                # d_out['Reference_valence'] =  get_cat_info(token_dicts, valence_ref, label)
                # d_out["Target's_role"] = get_cat_info(token_dicts, roles, label)
                # d_out['Group_membership'] = get_cat_info(token_dicts, group_ref, label)
                # d_out['Reference_demographics'] = get_cat_info(token_dicts, ref_dem, label)
                # d_out['Description_text'] = ''
                # d_out['Description_level'] = ''
                # d_out['Description_valence'] = ''
                # d_out['Description_group'] = ''
                # d_out['Description_demographics'] = '' 
                data_out.append(d_out)
                
                
               

        elif label not in label_coref_dict and label not in label_behav_mapping:
            print('found label not in coref dict', label)
            # print('Description only')
            d_out = dict()
            # d_out['Newspaper_id'] = news_id
            # d_out['Lstudio_id'] = ls_id
            d_out['Article_id'] = article_id
            # d_out['Topic'] = topic

            d_out['Referent_id'] = ''
            d_out['Reference_text'] = ''
            d_out['Reference_level'] = ''
#             d_out['Reference_valence'] = ''
#             d_out["Target's_role"] = ''
#             d_out['Group_membership'] = ''
#             d_out['Reference_demographics'] = ''

#             d_out['Description_text'] = tokens
#             d_out['Description_level'] = get_cat_info(token_dicts, lcm_level, label)
#             d_out['Description_valence'] = get_cat_info(token_dicts, valence_beh, label)
#             d_out['Description_group'] = get_cat_info(token_dicts, group_beh, label)
            # d_out['Description_demographics'] = get_cat_info(token_dicts, des_dem, label)

            data_out.append(d_out)

        


    return data_out
                


In [29]:

test_path = f'../results/kim-full-data-update1/output-5751477b947a3d0007b53c2c Klassiek jong txt_LS-id1334.csv'

article_id = 'output-5751477b947a3d0007b53c2c Klassiek jong txt_LS-id1334'
# topic = 'test'
# news_id = 'test'

#results/kim-full-data-update1/output-5751477b947a3d0007b53c2c Bridge jong txt_LS-id1333.csv

data_out = []

data_original = load_data(test_path)


data_out = process_file(test_path, ref_levels, article_id)

df = pd.DataFrame(data_out)

df[:30]

,Article_id,Referent_id,Reference_text,Reference_level
0,output-5751477b947a3d0007b53c2c Klassiek jong ...,1,de inwoners,high-TG-levels
1,output-5751477b947a3d0007b53c2c Klassiek jong ...,0,enkelen die behoefte hebben om naar klassieke ...,middle-TG-levels
2,output-5751477b947a3d0007b53c2c Klassiek jong ...,0,ze,middle-TG-levels
3,output-5751477b947a3d0007b53c2c Klassiek jong ...,0,Deze mensen,middle-TG-levels
4,output-5751477b947a3d0007b53c2c Klassiek jong ...,0,ze,middle-TG-levels


In [30]:
all_paths = os.listdir('../results/kim-full-data-update1/')

In [31]:
full_id_data = []
for path in all_paths:
    article_id = path.strip('output-').split('.')[0]
    full_path = f'../results/kim-full-data-update1/{path}'
 
    # print(full_path)
    if full_path.endswith('.csv'):
        id_data = process_file(full_path, ref_levels, article_id)
        full_id_data.extend(id_data)


found label not in coref dict SLTZMeMsB1
found label not in coref dict 0IeohiGIRF
found label not in coref dict QQMvKp7Ur-
found label not in coref dict meZGO35ryD
found label not in coref dict T5C0RwL3ZK
found label not in coref dict OfTqYW641d
found label not in coref dict BeBfI4npZy
found label not in coref dict t-KaTxIG8_
found label not in coref dict ndwG9TF_8o
found label not in coref dict ahJY0EKFJ9
found label not in coref dict 5yELOmbpI9
found label not in coref dict muOA-OtT03
found label not in coref dict D2g4o4k2OE
found label not in coref dict x5kaLbv-Wk
found label not in coref dict aUEKrCLli6
found label not in coref dict OMa_XcOdr3
found label not in coref dict do-nyNn4uC
found label not in coref dict ipCEbMdlWr
found label not in coref dict WrKZK2s72k
found label not in coref dict hzoOkVMZZh
found label not in coref dict sb3ezr3VHt
found label not in coref dict NzhL1u3be5
found label not in coref dict 0S97etDmAB
found label not in coref dict XKR7ufk3fK
found label not 

In [32]:
test_out = f'../results/kim-full-data-update1.xlsx'

df = pd.DataFrame(full_id_data)

df.to_excel(test_out, sheet_name='Sheet1', na_rep='', float_format=None, 
            columns=None, header=True, index=False)

/var/folders/y9/gpzq16dd7sd129brk7fnpzn00000gn/T/ipykernel_51507/3090032428.py:5: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel(test_out, sheet_name='Sheet1', na_rep='', float_format=None,


In [92]:
#df[:10]
#Co_12feb2023_P.21

In [36]:
df[:40]

,Article_id,Referent_id,Reference_text,Reference_level
0,inl5476578eb2e6-13656072110e Bingo oud txt,0,"alle bewoners , behalve de immer knorrige Harr...",middle-TG-levels
1,inl5476578eb2e6-13656072110e Bingo oud txt,0,de bewoners,middle-TG-levels
2,inl5476578eb2e6-13656072110e Bingo oud txt,0,elkaar,middle-TG-levels
3,inl5476578eb2e6-13656072110e Bingo oud txt,2,wie aanwezig is voor de gezelligheid,middle-TG-levels
4,inl5476578eb2e6-13656072110e Bingo oud txt,1,voor de competitie,middle-TG-levels
5,inl5476578eb2e6-13656072110e Bingo oud txt,1,Deze laatste groep,middle-TG-levels
6,inl5476578eb2e6-13656072110e Bingo oud txt,0,Het hele gezelschap,middle-TG-levels
7,inl5476578eb2e6-13656072110e Bingo oud txt,3,de meesten,middle-TG-levels
8,inl553232473742-137125854155 Bingo oud txt,1,Henk,individual-TG-levels
9,inl553232473742-137125854155 Bingo oud txt,2,Klaas,individual-TG-levels


In [ ]:
#         predicates = defaultdict(list)
#         for d in ds_unfolded:
#             pred = d['behav_chain_ids'].strip()
#             if pred != '':
#                 predicates[pred].append(d)
#         # print(predicates.keys())
                
#         refs = defaultdict(list)
#         for d in ds_unfolded:
#             ref = d['coref_chain_ids']
#             if ref != '':
#                 refs[ref].append(d)
#         # print(predicates.keys())
#         for pred, pred_ds in predicates.items():
#             pred_tokens = ' '.join([d['token'] for d in pred_ds])
#             # print(pred)
#             # print(pred_tokens)
#             if len(refs) > 0:
#                 for ref, coref_ds in refs.items():
                    
#                     pred_ids = [d['Behavior-predicate'] for d in coref_ds]

#                     # if pred in pred_ids:
#                     #     continue
#                     # else:
                    
#                      # check if valid span
                
#                     ref_tokens = ' '.join([d['token'] for d in coref_ds])
#                     target_ref_ids = set()
#                     for d in coref_ds:
#                         ref_ids = d['coref_chain_ids'].split(' ')
#                         for ref_id in ref_ids:
#                             coref_id, ref_id = ref_id.split('__')
#                             target_ref_ids.add(ref_id)
                        
#                     valid = False
#                     for ref_id in target_ref_ids:
#                         official_tokens = all_ref_tokens[ref_id]
#                         if ref_tokens == official_tokens:
#                             valid = True

#                     if valid:
                        
                        
                 
#                         # print('\t', coref_tokens)
#                         coref = [d['coref-relation']for d in coref_ds]

#                         d_out = dict()
#                         d_out['Newspaper_id'] = news_id
#                         d_out['Lstudio_id'] = ls_id
#                         d_out['Article_id'] = article_id
#                         d_out['Topic'] = topic

#                         d_out['Referent_id'] = coref
#                         d_out['Reference_text'] = ref_tokens
#                         d_out['Reference_level'] = get_cat_info(coref_ds, ref_levels)
#                         d_out['Reference_valence'] =  get_cat_info(coref_ds, valence_ref)
#                         d_out["Target's_role"] = get_cat_info(coref_ds, roles)
#                         d_out['Group_membership'] = get_cat_info(coref_ds, group_ref)
#                         d_out['Reference_demographics'] = get_cat_info(coref_ds, ref_dem)

#                         d_out['Description_text'] = pred_tokens
#                         d_out['Description_level'] = get_cat_info(pred_ds, lcm_level)
#                         d_out['Description_valence'] = get_cat_info(pred_ds, valence_beh)
#                         d_out['Description_group'] = get_cat_info(pred_ds,group_beh)
#                         d_out['Description_demographics'] = get_cat_info(pred_ds, des_dem)
#                         if d_out not in data_out:
#                             data_out.append(d_out)
#             else:
#                 d_out = dict()
#                 d_out['Newspaper_id'] = news_id
#                 d_out['Lstudio_id'] = ls_id
#                 d_out['Article_id'] = article_id
#                 d_out['Topic'] = topic
         
#                 d_out['Referent_id'] = ''
#                 d_out['Reference_text'] = ''
#                 d_out['Reference_level'] = ''
#                 d_out['Reference_valence'] =  ''
#                 d_out["Target's_role"] = ''
#                 d_out['Group_membership'] = ''
#                 d_out['Reference_demographics'] = ''
     
#                 d_out['Description_text'] = pred_tokens
#                 d_out['Description_level'] = get_cat_info(pred_ds, lcm_level)
#                 d_out['Description_valence'] = get_cat_info(pred_ds, valence_beh)
#                 d_out['Description_group'] = get_cat_info(pred_ds,group_beh)
#                 d_out['Description_demographics'] = get_cat_info(pred_ds, des_dem)
#                 data_out.append(d_out)
                
    
    
#     for coref, coref_ds in ref_only_dict.items():
        
#         coref_tokens = ' '.join([d['token'] for d in coref_ds])
                
#         d_out = dict()
#         d_out['Newspaper_id'] = news_id
#         d_out['Lstudio_id'] = ls_id
#         d_out['Article_id'] = article_id
#         d_out['Topic'] = topic
        
#         d_out['Referent_id'] = coref
#         d_out['Reference_text'] = coref_tokens
#         d_out['Reference_level'] = get_cat_info(coref_ds, ref_levels)
#         d_out['Reference_valence'] =  get_cat_info(coref_ds, valence_ref)
#         d_out["Target's_role"] = get_cat_info(coref_ds, roles)
#         d_out['Group_membership'] = get_cat_info(coref_ds, group_ref)
#         d_out['Reference_demographics'] = get_cat_info(coref_ds, ref_dem)
        
        
#         d_out['Description_text'] = ''
#         d_out['Description_level'] = ''
#         d_out['Description_valence'] = ''
#         d_out['Description_group'] = ''
#         d_out['Description_demographics'] = ''
#         data_out.append(d_out)


In [194]:
# ## BACKUP


# def get_cat_info(target_category_dicts, target_labels):
    
#     d = target_category_dicts[0]
    
#     labels = []
#     for l in target_labels:
#         val = d[l]
#         if val != '':
#             labels.append(l)
#     return ' '.join(labels)



# def unfold_data(data):

        
#     data_unfolded = []
#     data_unfolded_preds = []
    
#     for d in data:
        
#         ref_ids = d['coref-relation']
#         if ref_ids != '':
#             ref_ids = ref_ids.split(' ')
            
#             for rid in ref_ids:
#                 d_new = dict()
#                 d_new.update(d)
#                 d_new['coref-relation'] = rid
#                 data_unfolded.append(d_new)
#         else:
#             data_unfolded.append(d)
            
    
#     for d in data_unfolded:
#         pred_ids = d['behav-relation']
#         if pred_ids != '':
#             pred_ids = pred_ids.split(' ')
            
#             for prid in pred_ids:
#                 d_new = dict()
#                 d_new.update(d)
#                 d_new['behav-relation'] = prid
#                 data_unfolded_preds.append(d_new)
#         else:
#             data_unfolded_preds.append(d)
        
#     return data_unfolded_preds
    
    

# def process_file(test_path, topic, news_id, valence_ref, ref_levels, 
#                  roles, group_ref, ref_dem, lcm_level, valence_beh, group_beh, des_dem,):

#     id_data = []
    
#     try:
#         with open(test_path) as infile:
#             data = list(csv.DictReader(infile, delimiter = ','))
#     except FileNotFoundError:
#         print('file not found')
#         data = []
        
#     print(len(data))
#     data = unfold_data(data)
#     print(len(data))
        
          
#     ref_behav_dict = defaultdict(list)
#     coref_dict = defaultdict(list)
#     ref_only_dict = defaultdict(list)
    
#     for d in data:
#         rel_ref_behav = d['behav-relation']
#         rel_ref = d['coref-relation']
#         if rel_ref_behav != '':
#             for r in rel_ref_behav.split(' '):
#                 ref_behav_dict[r].append(d)
#         if rel_ref != '':
#             for r in rel_ref.split(' '):
#                 coref_dict[r].append(d)
                
#         if rel_ref != '' and rel_ref_behav == '':
#             for r in rel_ref.split(' '):
#                 ref_behav_dict[r].append(d)
        
    
#     data_out = []
#     for rel, ds in ref_behav_dict.items():
        
#         predicates = defaultdict(list)
#         target_categories = defaultdict(list)
#         for d in ds:
#             tcs = [i for i in d['Target-category-reference'].strip().split(' ') if i != '']
#             preds = [i for i in d['Behavior-predicate'].strip().split(' ') if i != '']
#             if tcs != []:
#                 for tc in tcs:
#                     target_categories[tc].append(d)
#             if preds != []:
#                 for pred in preds:
#                     if pred in predicates:
#                         if d not in predicates[pred]:
#                             predicates[pred].append(d)
#                     else:
#                         predicates[pred].append(d)

#         for tc, tc_ds in target_categories.items():
#             target_tokens = ' '.join([d['token'] for d in tc_ds])
#             ref_ids =  set()
#             if len(tc_ds) != 0:
#                 for tc in tc_ds:
#                     ref_ids.update([ref_id for ref_id in tc['coref-relation'].split(' ') if ref_id!= ''])
#             # print('target references')
#             # print(target_tokens, ref_ids)

#             if len(predicates) != 0:
#                 for pred_id, pred_ds in predicates.items():
#                     d_out = dict()
#                     d_out['Newspaper_id'] = news_id
#                     d_out['Lstudio_id'] = ls_id
#                     d_out['Article_id'] = article_id
#                     d_out['Topic'] = topic
#                     tokens = ' '.join([d['token'] for d in pred_ds])
#                     # print('\tpredicate')
#                     # print('\t', tokens)
#                     if len(target_categories) != 0:
#                         print(ref_ids)
#                         print(tokens)
#                         print(target_tokens)
#                         print()
#                         for ref_id in ref_ids:
#                             d_out['Referent_id'] = ref_id
#                             d_out['Reference_text'] = target_tokens
#                             d_out['Reference_level'] = get_cat_info(tc_ds, ref_levels)
#                             d_out['Reference_valence'] =  get_cat_info(tc_ds, valence_ref)
#                             d_out["Target's_role"] = get_cat_info(tc_ds, roles)
#                             d_out['Group_membership'] = get_cat_info(tc_ds, group_ref)
#                             d_out['Reference_demographics'] = get_cat_info(tc_ds, ref_dem)
#                             # ... 
#                             d_out['Description_text'] = tokens
#                             d_out['Description_level'] = get_cat_info(pred_ds, lcm_level)
#                             d_out['Description_valence'] = get_cat_info(pred_ds, valence_beh)
#                             d_out['Description_group'] = get_cat_info(pred_ds,group_beh)
#                             d_out['Description_demographics'] = get_cat_info(pred_ds, des_dem)
#                             data_out.append(d_out)

#                     else:
#                         d_out['Referent_id'] = ''
#                         d_out['Reference_text'] = ''
#                         d_out['Reference_level'] = ''
#                         d_out['Reference_valence'] =  ''
#                         d_out["Target's_role"] = ''
#                         d_out['Group_membership'] = ''
#                         d_out['Reference_demographics'] = ''
#                         # ... 
#                         d_out['Description_text'] = tokens
#                         d_out['Description_level'] = get_cat_info(pred_ds, lcm_level)
#                         d_out['Description_valence'] = get_cat_info(pred_ds, valence_beh)
#                         d_out['Description_group'] = get_cat_info(pred_ds,group_beh)
#                         d_out['Description_demographics'] = get_cat_info(pred_ds, des_dem)
#                         data_out.append(d_out)
#             else:
#                 # print(len(predicates), len(target_categories))
#                 ref_ids = set(ref_ids)
#                 # print(ref_ids)
#                 if len(target_categories) != 0:
#                     for ref_id in ref_ids:
#                         d_out = dict()
#                         d_out['Newspaper_id'] = news_id
#                         d_out['Lstudio_id'] = ls_id
#                         d_out['Article_id'] = article_id
#                         d_out['Topic'] = topic
#                         # print(tc_ds)
#                         d_out['Referent_id'] = ref_id
#                         d_out['Reference_text'] = target_tokens
#                         d_out['Reference_level'] = get_cat_info(tc_ds, ref_levels)
#                         d_out['Reference_valence'] =  get_cat_info(tc_ds, valence_ref)
#                         d_out["Target's_role"] = get_cat_info(tc_ds, roles)
#                         d_out['Group_membership'] = get_cat_info(tc_ds, group_ref)
#                         d_out['Reference_demographics'] = get_cat_info(tc_ds, ref_dem)
#                         # ... 
#                         d_out['Description_text'] = ''
#                         d_out['Description_level'] = ''
#                         d_out['Description_valence'] = ''
#                         d_out['Description_group'] = ''
#                         d_out['Description_demographics'] = ''
#                         data_out.append(d_out)

            
#     return data_out
  

